In [1]:
import pandas as pd
# https://www.avera.org/patients-visitors/price-transparency/

In [2]:
import csv
import subprocess
from urllib.parse import urlparse
import os

from dateutil.parser import parse as parse_datetime
import pandas as pd

In [3]:
TARGET_COLUMNS = [ 
    'hospital_id',
    #'row_id',
    'line_type',
    'description',
    'rev_code',
    'local_code',
    'code',
    'ms_drg',
    'apr_drg',
    'eapg',
    'hcpcs_cpt',
    'modifiers',
    'thru',
    'apc',
    'icd',
    'ndc',
    'drug_hcpcs_multiplier',
    'drug_quantity',
    'drug_unit_of_measurement',
    'drug_type_of_measurement',
    'billing_class',
    'setting',
    'payer_category',
    'payer_name',
    'plan_name',
    'standard_charge',
    'standard_charge_percent',
    'contracting_method',
    'additional_generic_notes',
    'additional_payer_specific_notes'
]

In [4]:
ccn = "161321"
url = "https://www.avera.org/app/files/public/887b57a5-04fc-4a68-9636-d6f0d9b2dfab/460224743_avera-merrill-pioneer_standardcharges.xlsx"
filename = url.split("/")[-1].replace("%20", " ")
ein = filename.split("_")[0]
ein = ein[:2] + "-" + ein[2:]

In [5]:
import httpx
import os

def create_session():
    session = httpx.Client(http2=True)
    
    session.headers = {
        'authority': 'www.avera.org',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
        'cache-control': 'no-cache',
        'pragma': 'no-cache',
        'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
    }
    
    resp = session.get("https://www.avera.org/patients-visitors/price-transparency/")
    print(resp.url)
    
    return session

def download_file(session, url, filename):
    if os.path.isfile(filename):
        print("{} already downloaded - skipping".format(filename))
        return
    
    resp = session.get(url)
    print(resp.url)
    
    out_f = open(filename, "wb")
    out_f.write(resp.content)
    out_f.close()
    
session = create_session()
download_file(session, url, filename)

https://www.avera.org/patients-visitors/price-transparency/
460224743_avera-merrill-pioneer_standardcharges.xlsx already downloaded - skipping


In [6]:
df_in = pd.read_excel(filename, skiprows=1, sheet_name='CDM')
df_in

,Facility Name,CPT,Charge Code,Gross Charge,Description,Hcpcs,Revcode,BCBS Commercial Negotiated Rate,Wellmark PPO Negotiated Rate,HealthPartners Commercial Negotiated Rate,PreferredOne PPO Negotiated Rate,Medica Commercial Negotiated Rate,UnitedHealthcare Commercial Negotiated Rate,Avera Health Plan Commercial Negotiated Rate,Discounted Cash Price,Minimum Negotiated Rate,Maximum Negotiated Rate
0,Avera Merrill Pioneer,77063,21L2364,79.0,Breast Tomosynthesis BI,NaN,0403,NaN,56.88,75.05,74.576,67.15,72.206,75.05,76.63,56.88,76.63
1,Avera Merrill Pioneer,72220,25L2085,454.0,Sacrum or Coccyx Min 2 View,NaN,0320,NaN,326.88,431.30,428.576,385.90,414.956,431.30,440.38,326.88,440.38
2,Avera Merrill Pioneer,NaN,0100025,1772.0,Room Medical Unit Semi Private,NaN,0120,NaN,1275.84,1683.40,1672.768,1506.20,1619.608,1683.40,1718.84,1275.84,1718.84
3,Avera Merrill Pioneer,NaN,0100027,1772.0,Room Medical Unit Priv,NaN,0111,NaN,1275.84,1683.40,1672.768,1506.20,1619.608,1683.40,1718.84,1275.84,1718.84
4,Avera Merrill Pioneer,NaN,0100048,680.0,Room Swing Bed Skilled,NaN,0129,NaN,489.60,646.00,641.920,578.00,621.520,646.00,659.60,489.60,659.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,Avera Merrill Pioneer,24600,9640424,803.0,Dislocation Elbow WO Anes,NaN,0981,NaN,578.16,762.85,758.032,682.55,733.942,762.85,778.91,578.16,778.91
1209,Avera Merrill Pioneer,25605,9640473,594.0,Fx Distal Radius W Man,NaN,0981,NaN,427.68,564.30,560.736,504.90,542.916,564.30,576.18,427.68,576.18
1210,Avera Merrill Pioneer,26770,9640547,368.0,Dislocation Finger WO Anes,NaN,0981,NaN,264.96,349.60,347.392,312.80,336.352,349.60,356.96,264.96,356.96
1211,Avera Merrill Pioneer,30901,9670909,283.0,Control Anterior Hemorrhage LT,NaN,0981,NaN,203.76,268.85,267.152,240.55,258.662,268.85,274.51,203.76,274.51


In [7]:
df_mid = pd.DataFrame(df_in)
df_mid = df_mid.rename(columns={
    'CPT': 'cpt',
    'Charge Code': 'local_code',
    'Description': 'description',
    'Hcpcs': 'hcpcs',
    'Revcode': 'rev_code'
})

columns = df_mid.columns.to_list()
columns.pop(columns.index('Gross Charge'))
df_mid = df_mid[columns + ['Gross Charge']]

del df_mid['Facility Name']

df_mid

,cpt,local_code,description,hcpcs,rev_code,BCBS Commercial Negotiated Rate,Wellmark PPO Negotiated Rate,HealthPartners Commercial Negotiated Rate,PreferredOne PPO Negotiated Rate,Medica Commercial Negotiated Rate,UnitedHealthcare Commercial Negotiated Rate,Avera Health Plan Commercial Negotiated Rate,Discounted Cash Price,Minimum Negotiated Rate,Maximum Negotiated Rate,Gross Charge
0,77063,21L2364,Breast Tomosynthesis BI,NaN,0403,NaN,56.88,75.05,74.576,67.15,72.206,75.05,76.63,56.88,76.63,79.0
1,72220,25L2085,Sacrum or Coccyx Min 2 View,NaN,0320,NaN,326.88,431.30,428.576,385.90,414.956,431.30,440.38,326.88,440.38,454.0
2,NaN,0100025,Room Medical Unit Semi Private,NaN,0120,NaN,1275.84,1683.40,1672.768,1506.20,1619.608,1683.40,1718.84,1275.84,1718.84,1772.0
3,NaN,0100027,Room Medical Unit Priv,NaN,0111,NaN,1275.84,1683.40,1672.768,1506.20,1619.608,1683.40,1718.84,1275.84,1718.84,1772.0
4,NaN,0100048,Room Swing Bed Skilled,NaN,0129,NaN,489.60,646.00,641.920,578.00,621.520,646.00,659.60,489.60,659.60,680.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,24600,9640424,Dislocation Elbow WO Anes,NaN,0981,NaN,578.16,762.85,758.032,682.55,733.942,762.85,778.91,578.16,778.91,803.0
1209,25605,9640473,Fx Distal Radius W Man,NaN,0981,NaN,427.68,564.30,560.736,504.90,542.916,564.30,576.18,427.68,576.18,594.0
1210,26770,9640547,Dislocation Finger WO Anes,NaN,0981,NaN,264.96,349.60,347.392,312.80,336.352,349.60,356.96,264.96,356.96,368.0
1211,30901,9670909,Control Anterior Hemorrhage LT,NaN,0981,NaN,203.76,268.85,267.152,240.55,258.662,268.85,274.51,203.76,274.51,283.0


In [8]:
money_columns = df_mid.columns[5:]
remaining_columns = df_mid.columns.to_list()[:5]
df_mid = pd.melt(df_mid, id_vars=remaining_columns, var_name='payer_name', value_name='standard_charge')
df_mid

,cpt,local_code,description,hcpcs,rev_code,payer_name,standard_charge
0,77063,21L2364,Breast Tomosynthesis BI,NaN,0403,BCBS Commercial Negotiated Rate,NaN
1,72220,25L2085,Sacrum or Coccyx Min 2 View,NaN,0320,BCBS Commercial Negotiated Rate,NaN
2,NaN,0100025,Room Medical Unit Semi Private,NaN,0120,BCBS Commercial Negotiated Rate,NaN
3,NaN,0100027,Room Medical Unit Priv,NaN,0111,BCBS Commercial Negotiated Rate,NaN
4,NaN,0100048,Room Swing Bed Skilled,NaN,0129,BCBS Commercial Negotiated Rate,NaN
...,...,...,...,...,...,...,...
13338,24600,9640424,Dislocation Elbow WO Anes,NaN,0981,Gross Charge,803.0
13339,25605,9640473,Fx Distal Radius W Man,NaN,0981,Gross Charge,594.0
13340,26770,9640547,Dislocation Finger WO Anes,NaN,0981,Gross Charge,368.0
13341,30901,9670909,Control Anterior Hemorrhage LT,NaN,0981,Gross Charge,283.0


In [9]:
df_mid['hcpcs_cpt'] = df_mid.apply(lambda row: row['cpt'] if row['cpt'] is not None else row['hcpcs'], axis=1)
del df_mid['cpt']
del df_mid['hcpcs']
df_mid

,local_code,description,rev_code,payer_name,standard_charge,hcpcs_cpt
0,21L2364,Breast Tomosynthesis BI,0403,BCBS Commercial Negotiated Rate,NaN,77063
1,25L2085,Sacrum or Coccyx Min 2 View,0320,BCBS Commercial Negotiated Rate,NaN,72220
2,0100025,Room Medical Unit Semi Private,0120,BCBS Commercial Negotiated Rate,NaN,NaN
3,0100027,Room Medical Unit Priv,0111,BCBS Commercial Negotiated Rate,NaN,NaN
4,0100048,Room Swing Bed Skilled,0129,BCBS Commercial Negotiated Rate,NaN,NaN
...,...,...,...,...,...,...
13338,9640424,Dislocation Elbow WO Anes,0981,Gross Charge,803.0,24600
13339,9640473,Fx Distal Radius W Man,0981,Gross Charge,594.0,25605
13340,9640547,Dislocation Finger WO Anes,0981,Gross Charge,368.0,26770
13341,9670909,Control Anterior Hemorrhage LT,0981,Gross Charge,283.0,30901


In [10]:
def payer_category_from_payer(payer):
    if payer == 'Gross Charge':
        return 'gross'
    elif payer == 'Discounted Cash Price':
        return 'cash'
    elif payer == 'Minimum Negotiated Rate':
        return 'min'
    elif payer == 'Maximum Negotiated Rate':
        return 'max'
    
    return 'payer'

df_mid['payer_category'] = df_mid['payer_name'].apply(payer_category_from_payer)
df_mid

,local_code,description,rev_code,payer_name,standard_charge,hcpcs_cpt,payer_category
0,21L2364,Breast Tomosynthesis BI,0403,BCBS Commercial Negotiated Rate,NaN,77063,payer
1,25L2085,Sacrum or Coccyx Min 2 View,0320,BCBS Commercial Negotiated Rate,NaN,72220,payer
2,0100025,Room Medical Unit Semi Private,0120,BCBS Commercial Negotiated Rate,NaN,NaN,payer
3,0100027,Room Medical Unit Priv,0111,BCBS Commercial Negotiated Rate,NaN,NaN,payer
4,0100048,Room Swing Bed Skilled,0129,BCBS Commercial Negotiated Rate,NaN,NaN,payer
...,...,...,...,...,...,...,...
13338,9640424,Dislocation Elbow WO Anes,0981,Gross Charge,803.0,24600,gross
13339,9640473,Fx Distal Radius W Man,0981,Gross Charge,594.0,25605,gross
13340,9640547,Dislocation Finger WO Anes,0981,Gross Charge,368.0,26770,gross
13341,9670909,Control Anterior Hemorrhage LT,0981,Gross Charge,283.0,30901,gross


In [11]:
df_mid['hospital_id'] = ccn
df_mid['line_type'] = None
df_mid['rev_code'] = df_mid['rev_code'].str[:4] # 0964an
df_mid['code'] = None
df_mid['ms_drg'] = None
df_mid['apr_drg'] = None
df_mid['eapg'] = None
df_mid['modifiers'] = ''
df_mid['thru'] = None
df_mid['apc'] = None
df_mid['icd'] = None
df_mid['ndc'] = None
df_mid['drug_hcpcs_multiplier'] = None
df_mid['drug_quantity'] = None
df_mid['drug_unit_of_measurement'] = None
df_mid['drug_type_of_measurement'] = None
df_mid['billing_class'] = None
df_mid['setting'] = None
df_mid['plan_name'] = None
df_mid['standard_charge_percent'] = None
df_mid['contracting_method'] = None
df_mid['additional_generic_notes'] = None
df_mid['additional_payer_specific_notes'] = None
df_mid

,local_code,description,rev_code,payer_name,standard_charge,hcpcs_cpt,payer_category,hospital_id,line_type,code,...,drug_quantity,drug_unit_of_measurement,drug_type_of_measurement,billing_class,setting,plan_name,standard_charge_percent,contracting_method,additional_generic_notes,additional_payer_specific_notes
0,21L2364,Breast Tomosynthesis BI,0403,BCBS Commercial Negotiated Rate,NaN,77063,payer,161321,None,None,...,None,None,None,None,None,None,None,None,None,None
1,25L2085,Sacrum or Coccyx Min 2 View,0320,BCBS Commercial Negotiated Rate,NaN,72220,payer,161321,None,None,...,None,None,None,None,None,None,None,None,None,None
2,0100025,Room Medical Unit Semi Private,0120,BCBS Commercial Negotiated Rate,NaN,NaN,payer,161321,None,None,...,None,None,None,None,None,None,None,None,None,None
3,0100027,Room Medical Unit Priv,0111,BCBS Commercial Negotiated Rate,NaN,NaN,payer,161321,None,None,...,None,None,None,None,None,None,None,None,None,None
4,0100048,Room Swing Bed Skilled,0129,BCBS Commercial Negotiated Rate,NaN,NaN,payer,161321,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13338,9640424,Dislocation Elbow WO Anes,0981,Gross Charge,803.0,24600,gross,161321,None,None,...,None,None,None,None,None,None,None,None,None,None
13339,9640473,Fx Distal Radius W Man,0981,Gross Charge,594.0,25605,gross,161321,None,None,...,None,None,None,None,None,None,None,None,None,None
13340,9640547,Dislocation Finger WO Anes,0981,Gross Charge,368.0,26770,gross,161321,None,None,...,None,None,None,None,None,None,None,None,None,None
13341,9670909,Control Anterior Hemorrhage LT,0981,Gross Charge,283.0,30901,gross,161321,None,None,...,None,None,None,None,None,None,None,None,None,None


In [12]:
df_mid = df_mid.dropna(subset=['standard_charge'])
df_mid

,local_code,description,rev_code,payer_name,standard_charge,hcpcs_cpt,payer_category,hospital_id,line_type,code,...,drug_quantity,drug_unit_of_measurement,drug_type_of_measurement,billing_class,setting,plan_name,standard_charge_percent,contracting_method,additional_generic_notes,additional_payer_specific_notes
1213,21L2364,Breast Tomosynthesis BI,0403,Wellmark PPO Negotiated Rate,56.88,77063,payer,161321,None,None,...,None,None,None,None,None,None,None,None,None,None
1214,25L2085,Sacrum or Coccyx Min 2 View,0320,Wellmark PPO Negotiated Rate,326.88,72220,payer,161321,None,None,...,None,None,None,None,None,None,None,None,None,None
1215,0100025,Room Medical Unit Semi Private,0120,Wellmark PPO Negotiated Rate,1275.84,NaN,payer,161321,None,None,...,None,None,None,None,None,None,None,None,None,None
1216,0100027,Room Medical Unit Priv,0111,Wellmark PPO Negotiated Rate,1275.84,NaN,payer,161321,None,None,...,None,None,None,None,None,None,None,None,None,None
1217,0100048,Room Swing Bed Skilled,0129,Wellmark PPO Negotiated Rate,489.60,NaN,payer,161321,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13338,9640424,Dislocation Elbow WO Anes,0981,Gross Charge,803.00,24600,gross,161321,None,None,...,None,None,None,None,None,None,None,None,None,None
13339,9640473,Fx Distal Radius W Man,0981,Gross Charge,594.00,25605,gross,161321,None,None,...,None,None,None,None,None,None,None,None,None,None
13340,9640547,Dislocation Finger WO Anes,0981,Gross Charge,368.00,26770,gross,161321,None,None,...,None,None,None,None,None,None,None,None,None,None
13341,9670909,Control Anterior Hemorrhage LT,0981,Gross Charge,283.00,30901,gross,161321,None,None,...,None,None,None,None,None,None,None,None,None,None


In [13]:
df_out = pd.DataFrame(df_mid[TARGET_COLUMNS])
df_out

,hospital_id,line_type,description,rev_code,local_code,code,ms_drg,apr_drg,eapg,hcpcs_cpt,...,billing_class,setting,payer_category,payer_name,plan_name,standard_charge,standard_charge_percent,contracting_method,additional_generic_notes,additional_payer_specific_notes
1213,161321,None,Breast Tomosynthesis BI,0403,21L2364,None,None,None,None,77063,...,None,None,payer,Wellmark PPO Negotiated Rate,None,56.88,None,None,None,None
1214,161321,None,Sacrum or Coccyx Min 2 View,0320,25L2085,None,None,None,None,72220,...,None,None,payer,Wellmark PPO Negotiated Rate,None,326.88,None,None,None,None
1215,161321,None,Room Medical Unit Semi Private,0120,0100025,None,None,None,None,NaN,...,None,None,payer,Wellmark PPO Negotiated Rate,None,1275.84,None,None,None,None
1216,161321,None,Room Medical Unit Priv,0111,0100027,None,None,None,None,NaN,...,None,None,payer,Wellmark PPO Negotiated Rate,None,1275.84,None,None,None,None
1217,161321,None,Room Swing Bed Skilled,0129,0100048,None,None,None,None,NaN,...,None,None,payer,Wellmark PPO Negotiated Rate,None,489.60,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13338,161321,None,Dislocation Elbow WO Anes,0981,9640424,None,None,None,None,24600,...,None,None,gross,Gross Charge,None,803.00,None,None,None,None
13339,161321,None,Fx Distal Radius W Man,0981,9640473,None,None,None,None,25605,...,None,None,gross,Gross Charge,None,594.00,None,None,None,None
13340,161321,None,Dislocation Finger WO Anes,0981,9640547,None,None,None,None,26770,...,None,None,gross,Gross Charge,None,368.00,None,None,None,None
13341,161321,None,Control Anterior Hemorrhage LT,0981,9670909,None,None,None,None,30901,...,None,None,gross,Gross Charge,None,283.00,None,None,None,None


In [14]:
df_out.to_csv(ccn + ".csv", index=False, quoting=csv.QUOTE_MINIMAL)